In [ ]:
pip install accelerate --upgrade


In [ ]:
 #pip install transformers[torch] accelerate>=0.20.1

In [ ]:
# pip install transformers[torch] accelerate -U

In [ ]:
# pip install accelerate -U

In [ ]:
# pip install --upgrade transformers

In [ ]:
# pip install -U huggingface_hub

In [ ]:
#pip install sacremoses

In [ ]:
import torch, os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from transformers import pipeline, TrainingArguments, Trainer
#from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
from torch import cuda
from sklearn.preprocessing import StandardScaler

from transformers import AlbertConfig, AlbertTokenizerFast, AlbertForSequenceClassification
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

if torch.cuda.is_available():
    # Print the name of the GPU device
    print('GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available. Switch to a GPU accelerator in Kaggle settings.')

In [ ]:
train_data = pd.read_csv('/kaggle/input/big-five-dataset/big_five_train_set.csv')
validation_data = pd.read_csv('/kaggle/input/big-five-dataset/big_five_val_set.csv')
test_data = pd.read_csv('/kaggle/input/big-five-dataset/big_five_eval_set.csv')

In [ ]:
print(train_data.shape)
print(validation_data.shape)
print(test_data.shape)

In [ ]:
train_data.columns.values

In [ ]:
from torch import nn

In [ ]:
# Define a custom dataset
class PersonalityDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        traits = torch.tensor(self.data.iloc[idx][['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']], dtype=torch.float)
        encoding = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length)
        input_ids, attention_mask = encoding["input_ids"].squeeze(), encoding["attention_mask"].squeeze()
        labels = data[traits.view(1, 5)]  # Reshape to (1, 5)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }


In [ ]:
tokenizer = AlbertTokenizerFast.from_pretrained("albert-base-v2",  max_length = 256)
config = AlbertConfig.from_pretrained('albert-base-v2', num_labels=5)
model = AlbertForPreTraining.from_pretrained("albert-base-v2", num_labels=5)

In [ ]:
num_regression_targets = 5  # Change this to match your number of regression targets
model

In [ ]:
#from transformers import AlbertModel, AlbertForSequenceClassification

num_regression_targets = 5

from transformers import AlbertModel, AlbertConfig
import torch.nn as nn

class CustomAlbertForRegression(nn.Module):
    def __init__(self, config, num_regression_targets):
        super(CustomAlbertForRegression, self).__init__()
        self.albert = AlbertModel(config)
        
        # Add your linear layers
        self.linear1 = nn.Linear(config.hidden_size, 128)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(128, 128)
        self.tanh1 = nn.Tanh()
        self.linear3 = nn.Linear(128, num_regression_targets)

    def forward(self, input_ids, attention_mask, token_type_ids=None, position_ids=None, head_mask=None, labels=None):
        outputs = self.albert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask
        )

        # Extract the last hidden state from the ALBERT model
        last_hidden_state = outputs.last_hidden_state

        # Calculate the mean of last_hidden_state across the sequence dimension
        pooled_output = torch.mean(last_hidden_state, dim=1)

        # Apply your additional linear layers
        linear_output = self.linear1(pooled_output)
        linear_output = self.relu1(linear_output)
        linear_output = self.linear2(linear_output)
        linear_output = self.tanh1(linear_output)
        regression_output = self.linear3(linear_output)

        if labels is not None:
            # Calculate the mean squared error loss if targets are provided
            loss = torch.nn.functional.mse_loss(regression_output, labels)
            return {'loss': loss, 'logits': regression_output}
        else:
            # Return predictions as a dictionary with "logits" key
            return {'logits': regression_output}


from transformers import AlbertTokenizerFast, AlbertConfig

tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v2', num_labels = 5)
config = AlbertConfig.from_pretrained('albert-base-v2', num_labels = 5)
num_regression_targets = 5
custom_model = CustomAlbertForRegression(config, num_regression_targets)


In [ ]:
custom_model.to(device)

In [ ]:
train_texts = list(train_data.text)
val_texts = list(validation_data.text)
test_texts = list(test_data.text)
train_labels = list(train_data[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
val_labels = list(validation_data[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)
test_labels = list(test_data[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']].values)

train_labels = np.array(train_labels, dtype=np.float32)
val_labels = np.array(val_labels, dtype=np.float32)
test_labels = np.array(test_labels, dtype=np.float32)

In [ ]:
val_labels.shape

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
print(train_encodings.keys())
print(len(train_encodings['input_ids']))
print(len(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0][:10])
print(train_encodings['attention_mask'][0][:10])

print(train_labels[0].dtype)


In [ ]:
class DataLoader(PersonalityDataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

In [ ]:

train_loader = DataLoader(train_encodings, train_labels)
val_loader = DataLoader(val_encodings, val_labels)
test_loader = DataLoader(test_encodings, test_labels)


In [ ]:
print(len(val_loader))  # Number of data points in the training dataset
print(len(val_loader[0]['labels']))  # Number of targets for the first data point
print(val_loader[0]['labels'])


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 5)
    print("Labels shape:", labels.shape)
    print("Logits shape:", logits.shape)
    print(labels)

    # Calculate regression metrics
    mse = mean_squared_error(labels, logits, multioutput='raw_values')
    mae = mean_absolute_error(labels, logits, multioutput='raw_values')
    r2 = r2_score(labels, logits, multioutput='raw_values')
    # Compute accuracy 
    rmse = np.sqrt(mse)
    single_squared_errors = ((logits - labels)**2)
    accuracy = [sum(e < 0.25) / len(e) for e in single_squared_errors.T]

    # Calculate the mean of all metrics
    metrics = {
        "mse": mse.mean(),
        "rmse": rmse.mean(),
        "mae": mae.mean(),
        "r2": r2.mean(),
        "accuracy": sum(accuracy) / len(accuracy)
    }
    
    return metrics





In [ ]:
#base_model = "albert"
learning_rate = 2e-5
max_length = 256
batch_size = 4
epochs = 3

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./models/albert-fine-tuned-regression",
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_gpu_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="mse",
    load_best_model_at_end=True,
    weight_decay=0.01
)

In [ ]:
import torch


class RegressionTrainer(Trainer):

    def compute_loss(self, custom_model, inputs, return_outputs=False):
        try:
            labels = inputs.pop("labels", None)
            print("Labels:", labels)
            if labels is not None:
                outputs = custom_model(**inputs)
                logits = outputs.logits
                loss = torch.nn.functional.mse_loss(logits, labels)
                return (loss, outputs) if return_outputs else loss
            else:
                print("Labels are None in compute_loss")
                return None
        except Exception as e:
            print("Error in compute_loss:", e)
            print("Inputs:", inputs)
            return None


In [ ]:
custom_model.to("cuda:0")


In [ ]:
num_regression_targets = 5  # Change this to match your number of regression targets

trainer = Trainer(
    model=custom_model,
    args=training_args,
    train_dataset=train_loader,
    eval_dataset = val_loader,
    compute_metrics = compute_metrics_for_regression
)


In [ ]:
# Before training, inspect the evaluation dataset
for i, data in enumerate(val_loader):
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    labels = data['labels']
for i, data in enumerate(train_loader):

    # Print shapes and labels for debugging
    print(f"Sample {i + 1} - Input IDs shape:", input_ids.shape)
    print(f"Sample {i + 1} - Input IDs :", input_ids)
    print(f"Sample {i + 1} - Attention Mask shape:", attention_mask.shape)
    print(f"Sample {i + 1} - Attention Mask:", attention_mask)
    print(f"Sample {i + 1} - Labels shape:", labels.shape)
    print(f"Sample {i + 1} - Labels values:", labels)
    print(f"Sample {i + 1} - Labels values:", data['labels'])
    break
# ... Rest of your code ...

In [ ]:
trainer.train()

In [ ]:


# Evaluate on validation dataset
eval_result = trainer.evaluate(val_loader)

print("Evaluation result:")
for key, value in eval_result.items():
    print(f"{key}: {value}")


In [ ]:
# New text input
new_text = 'I hate to go out and party'

# Tokenize the input text
encoding = tokenizer(new_text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)

# Move the input tensors to the GPU (cuda:0)
input_ids = encoding["input_ids"].to("cuda:0")
attention_mask = encoding["attention_mask"].to("cuda:0")

# Move the model to the GPU (cuda:0)
model.to("cuda:0")

# Pass the tokenized input through the model to obtain predictions
outputs = model(input_ids, attention_mask=attention_mask)

# Get the predicted scores
predicted_scores = outputs.logits.squeeze().tolist()

print("Predicted Scores:", predicted_scores)


In [ ]:
# Define the labels (traits)
labels = ['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']

# Get the predicted scores
predicted_scores = outputs.logits.squeeze().tolist()

# Create a dictionary to associate the labels with the predicted scores
predicted_scores_dict = dict(zip(labels, predicted_scores))

print("Predicted Scores:")
for trait, score in predicted_scores_dict.items():
    print(f"{trait}: {score}")
